In [37]:
import os

os.getcwd()

'E:\\Chicken-Desease-Classification\\Chicken-Desease-Classification'

In [38]:
working = "E:\\Chicken-Desease-Classification\\Chicken-Desease-Classification"
os.chdir(working)
os.getcwd()

'E:\\Chicken-Desease-Classification\\Chicken-Desease-Classification'

In [39]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoinnt_model_filepath: Path

In [40]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [41]:
class ConfigurationManager():
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [42]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [43]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [44]:
import os
import time
from tensorflow import keras

class PrepareCallbacksConfig:
    def __init__(self, tensorboard_root_log_dir, checkpoint_model_filepath):
        self.tensorboard_root_log_dir = tensorboard_root_log_dir
        self.checkpoint_model_filepath = checkpoint_model_filepath

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    def _create_ckpt_callbacks(self):
        return keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks(),
            self._create_ckpt_callbacks()
        ]

In [45]:

try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

[2024-06-28 23:50:14,157: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-28 23:50:14,159: INFO: common: yaml file: params.yaml loaded successfully]
[2024-06-28 23:50:14,160: INFO: common: created directory at: artifacts]
[2024-06-28 23:50:14,161: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-06-28 23:50:14,162: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
